# 进展记录
该脚本主要记录实验进展，包括但不限于实验设置、实验结果、分析、疑惑记录等内容

# 11月

## 11.26
### 记录
`logs/rl_games/leaphand_object_rot/2025-11-26_10-28-35` 关节空间
<br> `logs/rl_games/leaphand_se3_rot/2025-11-26_10-03-32` se3末端刚体旋量空间
<br> `logs/rl_games/leaphand_se3_rot/2025-11-26_21-48-16` se3缩放修正版
#### 分析与疑惑
- 试验了se3动作空间的训练效果，初步结果不太理想。主要问题在于奖励收敛很慢，且最终效果不如关节空间动作。怀疑原因有：1是se3动作空间的高维度和冗余性导致训练难度加大；2是当前的奖励设计和观察空间并未针对se3动作进行优化，导致学习效率低下。
- 先做了简单的对比训练，除了动作项外其他都一致。se3动作能做，但奖励收敛很慢。同时发现了一些奇怪的现象：1是se3手指和物体不怎么接触，很多是靠指腹带动物体转；2是实际关节动作抖动性不算太大，但经常一阵一阵的反复，不知道是se(3)旋量和J_b的伪逆导致的这种不稳定行为不。
- 传统的EMA，是在神经网络外部显示裁减增量幅度来达到动作平滑化，并非网络内生平滑。但我目前如果在深度学习这块去做，不知道来不来得及，但实现神经网络的物理内生一致性，达到对于大多数任务所需要的平滑一致动作输出，确实是个潜在的idea
- 虽然初次训练效果一般，但我仍坚信se(3)的应用潜力。也许每个动作表征空间都有自身适用的机制，它们的偏好分布具有偏差，强制相同条件下去比对也未必公平。或许当下阶段，不是在当前关节空间最合适的配置里去调se3旋量动作空间的效果，而是从se3本身出发，去调整适配于se3自身的观察、奖励等，以达到效果更好。真正的比较应该是SOTA比SOTA，关节空间SOTA比se3旋量空间SOTA
- 非常奇怪，修改了se3actions.py里的缩放，但训练效果反而更差了，不如之前的版本。怀疑是哪里有bug，待排查
> 更正：缩放修正后后期奖励比未修正的要好，最后阶段的奖励是未修正时的4倍左右，可能只是前面慢一点；
### 计划
开始做针对se3的适配工作，分为5大块
1. 设置适合se(3)的观察空间。
<br>感觉关节量可能不是很合适，因为这相当于仍要学习一个正向运动学的过程构建，网络会隐式地包含手的结构信息。但本体感知的信息本身就很少，关节量适合用作基准参照。
难道说，这时的本体感知变成了目标旋量动作？
理想中更合适的相对信息，来自视触觉模态信息。一种是点云，手和物体点云所构成的相对信息。一种是触觉，如视触觉。
重点可探索点云方面的视觉信息，最好是即时可用的。
IDEA：不是说结构信息不必学，是想让策略网络更关注任务本身的层面。结构这块的信息，或许可以参照GET-Zero训练一个上游特征的结构提取基座大模型G: q -->q* ，但怎么去理解手的结构本身就是一个难题
1. 奖励函数设计的调整 (Crucial for se3)
<br>关于动作稳定和平滑，可以改为惩罚SE(3)能量 ${\mathcal{V}_b^{T}}\mathcal{G}\mathcal{V}_b$ ，以及 。
关于操作多样性，可用可操作度指标 (Manipulability Index) $det(J{{J}^{T}})$ 或 雅可比条件数 $\frac{{{\sigma }_{max}}(J)}{{{\sigma }_{min}}(J)}$ 。
<br>另外可以考虑触觉方面的奖励，这个可能要调用IsaacLab的触觉传感器
1. 动作的微调
<br>先想的是额外再添加一个动作缩放因子  
可选：自适应 $\lambda$，具体实现方式待定；PD控制中引入位置误差补偿（Closed-Loop Inverse Kinematics, CLIK），有无必要确认：
1. 网络的设置
<br>现有的动作维度为24，以及考虑到如果用关节作为本体感知信息，那终究还是要隐式的学习手的动力学结构，而手指为4个节点的树，那是否意味着用4层MLP会更好？还有动作空间匹配的问题
1. 学习算法
<br>不知道rl_games最新的pbt训练是否对训练效果的提升或者超参数的调整有所增益？SAC算法和TD3算法和PPO相比又如何

## 11.27

### 计划与行动
今天主要针对奖励这块修正。昨天的实验，根据奖励管理器的记录，我觉得pose_diff权重偏高，而且其设置的默认位形可能比较奇异，我感觉是不太好的。关于pose_diff，我可以重新人为设置和调整下更合适的初始位形，甚至有一个
IDEA：利用别人预训练好的抓取网络，生成一个抓取姿态反解得到初始位形（这样可以保证手指和物体有良好的接触、且可针对不同物体进行泛化）。后期训练过程中可以通过课程学习逐渐降低pose_diff的权重。

1. 奖励的设计与调整
<br>主要想调整pose_diff和操作度指标这两块，思路和昨天设想的一致。
<br>
2. 可操作度分析
<br> `source/leaphand/leaphand/tasks/functional/test_manipulability.py`。对操作度、雅可比条件数的计算和显示进行了修正，添加了打印按钮，可以将当前各手指的雅可比矩阵和奇异值信息打印到终端，便于分析。
- 从结果来看，条件数普遍较大（>50)，操作度都很小(<0.1)，这可能是每根手指自由度数量较少的缘故（4个自由度），导致雅可比矩阵接近奇异矩阵。
- LeapHand 是一个 4-DOF 手指系统，需要在 6D 空间（SE(3)）操作。这是一个欠驱动系统 (m=6 > n=4)。即使在最佳位形，手指也无法实现任意 6D 运动（只能实现 4D 子空间的运动）。高条件数和低可操作度是欠驱动系统的固有特性，不一定是错误
- 所以，该如何设计奖励呢？这是一个问题啊，操作度和条件数变化幅度较大。难道也是从课程学习的角度，一开始奖励操作度较高的、鼓励探索，后面降低权重？因为完成任务本身和位形对应的操作度可能并无直接关联
- IDEA：从任务角度来说，我觉得末端的三位位置相对更重要些，同时因为4-Dof，导致位置和姿态高度耦合，所以学习的时候，如果能学到比较好的位置、姿态一样也会比较好。这一过程该如何数学建模呢，只关心旋量的平移部分，让学习的维度降下来？
- NOTE：如果想反映位置的重要性，可用位置可操作度，$J_{\text{pos}} \in \mathbb{R}^{3 \times 4}$，即只取雅可比矩阵的前三行（Physx里前三行是线速度）来计算可操作度和条件数，或许能更好地反映位置控制能力；或者适用加权雅可比矩阵，通过单位换算和权重调整，人为地“缩小”姿态空间的重要性，从而让奇异值分解（SVD）主要反映位置能力，同时兼顾姿态。既保持位置灵活性，又防止整体奇异。

## 11.28


### 计划
1. 先回顾一下 `Velocity Kinematics and statics`
2. 继续调整奖励函数，并争取当晚能继续做一组对比实验
### 问题
1. 是否可以从urdf或usd文件获取手指的解析雅可比矩阵？如果能够获取，感觉可以加速计算，或者让我对手的操作性有个更直观的认知。
> 获取了感觉也没太大用，计算实际用不到，而且误差较大。
> 但是我们可以通过urdf的解析雅可比矩阵，做可操作性的理论分析，给我们直观认知。特别是手指末端的位置雅可比及操作度
2. NOTE：除了操作度椭球，还有力矩椭球。两者在相同位形上是正好成反比的，操作度椭球在某个方向上越大，力矩椭球在该方向上就越小。该如何衡量呢？


# 12月

## 12.2

### 计划
1. 不用想太多，明确操作度或条件数的用途，添加相应的奖励
2. 新思路，加权雅可比矩阵
### 问题
1. 新加了加权雅可比矩阵在 `source/leaphand/leaphand/tasks/functional/test_manipulability.py`。但是我发现在滑动每根手指最后一个关节时，该手指的雅可比矩阵并为变化，这让我有极大的怀疑，手指末端不是在指尖（tip_head)，而是tip，所以最后一个自由度对末端位置没有影响，导致雅可比矩阵不变。自然，我们分析的操作度等度量是要反映我们定义的指尖的，可通过 $J_{b'}=\mathrm{Ad}_{T_{b'b}}J_b$ 获取
2. 打算再次重构 `test_manipulability`，并完善 `source/leaphand/leaphand/tasks/manager_based/leaphand/mdp/utils/math.py` 和采用里面的计算

## 12.3

### 计划
1. 重构 `test_manipulability`
2. `se3_actions.py` 也有问题
### 重构与新增
核心思路已经在笔记里完成，明天在各文件里进行具体实现

## 12.4


### 事项
1. 已重构 `test_manipulability` 和 `se3_actions.py`，修正了雅可比矩阵的逻辑错误问题
2. 着手安排 se3wdlsAction, se3adlsAction, se3awdlsAction 的实现工作
3. 该几个动作项可和奖励、观察联动进行实现
### 记录
Tikhonov正则化和dls是同一种方法的不同叫法，都是通过在伪逆计算中添加阻尼项来稳定解。Tikhonov正则化更强调数学上的正则化视角，而DLS更侧重于机器人学中的应用。它们本质上是相同的技术，只是名称和侧重点不同。

## 12.5


### 目标
完成其余动作项的实现，以及相应的观察项和奖励项
### 记录
可操作度椭球和力椭球对偶，但还是先从可操作度椭球做起

## 12.7
### 记录
- 目前数值稳定性严重影响了训练效果，效果甚至不如最开始的se3dlsAction，有点失望了，陷入到工程里的打磨了，不知道出路在哪
### 事项
- 重构了se3_actions.py，加权自适应阻尼等项已经实现。同时伪dls计算由cholesky分解改为了svd分解
### 念想
- 刚开始真没必要一口气推进这么多变体搞得太快，感觉现在做对比、观察哪块对训练效果有提升都变得困难

## 12.8

## 记录
1. 三组比对实验：
> `AnyRotate/logs/rl_games/leaphand_se3_rot/2025-12-08_02-21-04` W_x = [1,1,1,5,5,5] horizon_length=32, minibatch_size=800 奖励到达了31
> <br> `AnyRotate/logs/rl_games/leaphand_se3_rot/2025-12-08_04-09-02` W_x = [1,1,1,5,5,5] horizon_length=48, minibatch_size=1200 奖励到达了41
> <br> `AnyRotate/logs/rl_games/leaphand_se3_rot/2025-12-08_04-13-30` W_x = [1,1,1,1,1,1] horizon_length=48, minibatch_size=1200 奖励到达了78
  - W_x为单位阵时反倒比W_x加权时效果好
  - 稍微增大一点 `horizon_length` 和 `minibatch_size` 带来了相同训练步数下奖励的提升
  - 但都仍不及关节空间表现，甚至不如先前的dls
  - 有可能是 `pose_diff` 奖励取消了的缘故
  - `AnyRotate/logs/rl_games/leaphand_se3_rot/2025-12-08_04-13-30/nn/leaphand_se3_rot.pth` 还有持续训练的必要性，因为奖励还远远未达到顶峰，估摸着可以把max_epochs提升至20000
2. 不清楚SVD分解-自适应阻尼的行为，需要review

## 设计
1. 想要再设计三组对比实验（1组基准组 + 2组对比组），用于验证 `pose_diff` 和 动作项 `svd`分解的正负优化效果
> 目前实验进展：我发现加了 `pose_diff` 的反而训练没那么好，有可能是关节索引问题，待排查。后续再奖励项中改为字典设置型的，无论何种索引都保证一定能和实际关节对应上
2. 准备补一个新的任务：手内重定向

## 12.9

### 记录
1. 三组对比实验
> 先前最好 `AnyRotate/logs/rl_games/leaphand_se3_rot/2025-11-26_21-48-16/params/env.yaml` 同样是固定阻尼，它这个是se3dlsactions（但那个时候底层算法和现在有所不同），有 `pose_diff` ，奖励达到了176
> <br>`AnyRotate/logs/rl_games/leaphand_se3_rot/2025-12-08_21-17-13/params/env.yaml`  基准，奖励来到了86
> <br>`AnyRotate/logs/rl_games/leaphand_se3_rot/2025-12-08_21-18-27/params/env.yaml` ➕ `pose_diff` 11.3,居然负优化了
> <br>`AnyRotate/logs/rl_games/leaphand_se3_rot/2025-12-08_21-23-29/params/env.yaml` ➕ `pose_diff` + `选择性自适应阻尼` 9.52, 差不多负优化...
- `pose_diff`负优化了，但真的是这样吗
- `dls_inv`也负优化了
### 行动
先不急着改进，先回到过去的黄金版本，是否能复现那个时候的奖励呢

## 12.10

### 记录
1. 两组对比实验
> `AnyRotate-temp/logs/rl_games/leaphand_se3_rot/2025-12-09_15-20-09/nn/leaphand_se3_rot.pth`  原始版本，无自适应阻尼，雅可比是几何雅可比，参考点在{b}，参考系在{w}，这样训练出的仍是刚体旋量Vb，但它是表达在{w}，奖励达到了760
> <br>`AnyRotate-temp/logs/rl_games/leaphand_se3_rot/2025-12-09_16-10-49/nn/leaphand_se3_rot.pth`  对比版本，依然是固定阻尼，雅可比是刚体雅可，参考点在{b}，参考系也在{b}，是点位一体的刚体旋量Vb，但奖励只有428
- 从这样看，可能位于Vb参考系在{w}时，网络更容易学习到有效的动作策略，从而提升了训练效果，因为相同的输出，在动作效果和物理意义想要比较明确，相对于环境坐标系来说
- 但无论如何，se3训练的效果都很差，远不如关节空间上进行训练
2. 补充对比（观察空间）
> `AnyRotate/logs/rl_games/leaphand_se3_rot/2025-12-10_15-59-38` 原始 + 实际归一化旋量观察 奖励182
> <br>`AnyRotate/logs/rl_games/leaphand_se3_rot/2025-12-10_15-59-38` 原始 奖励239
- 加入了实际旋量，反而恶化了训练效果（但程度不高），可能信息过载了。
- 和temp相比，可能奖励设置的不同也导致239低于428，但效果都很差，不如关节空间
### 计划
1. 合并代码。 把 `AnyRotate-temp` 和 `AnyRotate` 整理后合
<br>以下代码很可能是训练效果惨淡的来源
```python
        # 5. 获取目标刚体在世界坐标系下的位姿
        body_pos_w = self._asset.data.body_pos_w[:, self._body_idx]  # shape: (num_envs, 3)
        body_quat_w = self._asset.data.body_quat_w[:, self._body_idx]  # shape: (num_envs, 4)
        
        # 6. 构造从世界坐标系到刚体坐标系的变换矩阵
        T_wb = math_leap.transform_from_pos_quat(body_pos_w, body_quat_w)  # shape: (num_envs, 4, 4)
        T_bw = math_leap.inverse_transform(T_wb)  # shape: (num_envs, 4, 4)
```
body_pos_W所导致

2. 数据集构建和模仿学习。实行 Teacher（Joint Space) —— Student（se3 Space) 的策略蒸馏
3. 【可选】执行idea.ipynb中的第一种思路

## 12.11

### 计划
1. 了解 Recorder Manager。并重构管理项
2. 进行 BC 训练

## 12.17

### 进展
1. 模仿学习在 Joint -> se3 的表述建构
2. `recorders.py` 及 `recorders.py` 大部分实现。但仍存在一定问题，包括固定参数组的解耦，还有se3action动作访问的掺杂。需要review并明确返回值和数据集中数据项构建的对应关系

## 12.18

### 计划
1. Review recorders.py 和 recorders_cfg.py
2. 解决昨晚发现的问题：se3动作项耦合；剔除部分不需要记录的元数据；
3. 弄清楚记录的机制

## 12.22

### 记录
1. self.robot.spawn.articulation_props.fix_root_link = False # 让基座浮动
   使得 `random_agent.py` 的随机动作能让基座浮动起来

## 12.24

## 想法
得趁资源空闲这段时间，做一些消融对比实验，完善工程方面的trick
1. 历史步的长度和mlp
2. 特权信息编码器
3. 辅助目标的旋转奖励
4. 稳定抓取环境任务
5. 触觉传感
6. 旋转不同的物体
7. 球形指尖和原始的表现差异
8. 自动域随机化ADR
9. minibatch_size大小对训练的影响

## 12.25

## 记录
1. 球型指尖和人手型指尖对比
> `AnyRotate/logs/rl_games/leaphand_object_rot/2025-12-24_23-31-14/nn/leaphand_object_rot.pth` 球型指尖：最高奖励844左右，动作看起来也还好，但方块经常位于掌根，怀疑是初始位置设置不合理
> `AnyRotate/logs/rl_games/leaphand_object_rot/2025-12-24_23-29-05/nn/leaphand_object_rot.pth` 人手型指尖：最高奖励1427左右，仍是目前的SOTA，方块位于手掌中心

感觉暂时无法说明哪个指尖更优
<br>经额外的对比实验组，发现物体初始位置对奖励和训练表现的影响尤为大，很可能是碰撞行为所导致的